This is an educational simulation to demonstrate how to implement the main concepts of [probabilistic payment delivery with the additional maintainance of the uncertainty network](https://arxiv.org/abs/2103.08576) and [optimal splitting of payments in MPP which is also known as optimally realiable payment flows](https://arxiv.org/abs/2107.05322)

This code assumes an oracle with the actual liquidity of channels to exist. In reality the oracle is the lightning network which can be queried by sending out actual onions.

## Warning: THIS IS WORK IN PROGRESS: The code has some severe limitations...
...which make it impractical for real nodes and implementations. Do not just blindly wrap this into production code but address the following limitations:

* channel reserves are ignored
* HTLC limits like min_size, max_size, max_accepted_htlc are fully ignored
* proper handling of parallel channels needs to be done
* strategies to operate with hanging htlc's (some might require protocol updates) may need to be included
* concurrent handeling of onions and maintanance of the uncertainty network
* proper handling of non zero base fee channels is needed and in particular handeling their cost properly at least as good as it is possible
* better piecewise splitting / quantization of the amounts is necessary
* learning how to weight various features needs to be conducted or at least configureably be in place
* Mechanics of making learnt information persistant over some time
* inclusion of other otpimization goals that predict reliability are missing (like latency, distance, ...)
* One should Prune the graph before invoking the solver
* properly conduct the piecewise linearization of the cost function
* don't overwrite memory of the entire uncertainty network all the time but just update the necessary pieces before invoking the solver
* potentially write your own stand alone cost scaling min cost flow solver instead of relying on a third party dependency


## BOLT14 considerations

This code also contains a few experiments to investigate the effects of the BOLT14 proposal.
Maybe it might make sense to make entropy hints not only as a foaf query but as an n-bit network wide gossip message. Unless we have some prior belief about the channel the 2 bit gossip intervalls are

* [0, c/4]
* [c/4,c/2]
* [c/2,3c/4]
* [3c/4,c]

nodes would only signal a message if the liquidity switches from one of those intervals to another (of course one could also require 3 bits or so)

## TODOS: 
as hinted in the ldk issue https://github.com/lightningdevkit/rust-lightning/issues/1372 it might make sense to not use a global \mu but rather learn it for each channel by comparing min uncertainty cost ~ 1/c max uncertainty cost = log(c+1) with the ppm on the edge and make sure we get this in the same order. It is only a test but it might turn out to work really well

fix some fixme's most notably the data model that produces poor probability computation in outputs as it ignors the prior knowlege which means all displayed probabilities are too low (though the flow is computed properly)

make a useful design for liquidity sharing...

# Funding

This research result is funded through [NTNU](https://www.ntnu.no) and [BitMEX](https://blog.bitmex.com/bitmex-2021-open-source-developer-grants/) as well as generous donations from the Bitcoin community via https://donate.ln.rene-pickhardt.de and via recurring donations at https://www.patreon.com/renepickhardt. If you want to learn why independent research and development for Bitcoin and the Lightning Network is important I kindly refer you to: https://ln.rene-pickhardt.de/ of course I will be grateful if you consider my work of importance and decide to support it.

Special Thanks to Stefan Richter with whom we implemented the first mainnet tests on top of lnd which certainly helped me to clarify many details and get the maintanance of the uncertainty network straight.

# Glossary of terms

As many similar terms are being used for similar concepts and things and since I have been studying these topics for three years now I will follow the good practice of [BOLT0](https://github.com/lightning/bolts/blob/master/00-introduction.md) and put a glossary of used terms up here with the hope that I finally have some good, useful and precise defintions that will also bee picked up by others.


## (Payment) Channel
Payment `channel`s are either announced or unnannounced but known to the sender and recpient of a payment. They have meta data like the `capacity` or `routing fees` as well as some other config values which we ignore for simplicity here.

##  Arc
An `arc` is not to be confused with a payment `channel`. `Arc`s are the data structure that arise from the piece wise linearized `cost function` and are used internally in the min cost flow solver. 


## (Channel) Capacity
The `capacity` of a payment channel is its size as observed in the amount of the output of the funding transaction which is announced via the `short_channel_id` in gossip.

**Important**: It seems to be an unavoidable collision that the term capacity is also used for `arcs` in min cost flow solvers. As I introduce the piece wise linearization the `cost function` the arcs in the min cost flow solve will usually have a lower `capacity` than the cannels they represent. I try to distinguish this by talking either about the `channel capacity` or the `arc capacity` if absolutely necessary.

## Liquidity
The `Liquidity` is the amount of satoshis a peer has available to route on it's channel. This means that unless the (even) `capacity` is split exactly 50/50 between the peers the `liquidity` in a channel will be different for both peers. 

typical terms in the wild: balance, local balance, outbound capacity

## Uncertainty / Entropy 
assuming a prior probability distribution we can quantify the `uncertainty` about the channels `liquidity` by measureing the `entropy` of the probability function. Notably if a channel as a `capacity` of `c` satoshis the entropy is `log(c+1)`

## Unertainty Network

The `uncertainty network` is a model than encodes our current belief about the available `liquidity` in the channels of the Lightning network. This is usually achieved by storing three values: 

1. Minimum `liquidity` which initialy for remote channel is `0`. 
2. Maximum `liquidity` which initially for remote channels corresponds its `capacity`.
3. `inflight` is the number of satoshis that we currently have allocated to this channel in outstanding onions and HTLCs.

## inflight
We use `inflight` to encode how many satoshis we have currently allocated or plan to allocate to a channel. Note that this is not the same as the number of inflight htlcs that the peer co-owning the channel will observe as some of the onions that we have outstanding may not have been delivered to the channel yet or may have failed but not returend back to us yet.

## Cost(function)

The `cost` function encodes our optimization goals and consists of several `features`. typical `features` are the `uncertainty cost` and the `routing fees`. Note that the same cost function can be applied to Dijkstra's algorithm as to min cost flow solvers. However in the general case of min cost flows the cost function should be convex or even better linear or piecewise linear so that a reasonable runtime can be achieved. 

The `cost` is just an internal value that will be minimized in computation of `candidate paths` for the `payment loop`. This cost may or may not translate to `routing fees`.

typical terms in the wild: Score, fees, penalty

## Unit cost
Linear min cost flow solvers need a fixed cost for transporting 1 unit (e.g. 1 satoshi) along an `arc` (e.g. `channel`). Oft

## Uncertainty cost
This is the `feature` introduced in the [`probablistic payment delivery` paper](https://arxiv.org/abs/2103.08576) it puts a cost on an amount `a` to be send over a (`path`) of `payment channels`.
It is comuted by taking the negative logarithm of the `success probability` if we have no prior belief of the channel the `uncertainty cost` will be computed as: `-log((c+1-a)/(c+1))`. Note that the uncertainty cost is a convex gowing function takin the value `0 = -log((c+1-0)/(c+1)) = -log(1)` if no sats are to be allocated on the channel. The maximum possible `uncertainty cost` occurs if `a=c` and takes the value `log(c+1)` which is exactly the `entropy` of the channel which can be seen by the following calculation:
`-log((c+1-c)/(c+1)) = -log(1/(c+1)) = log(c+1)`

### linearized uncertainty cost
The linearized uncertainty cost is the first term of the taylor approximation of `f(a)=-log((c+1-a)/(c+1))` which results in `a/(c+1)`. This (in my opinion surprisingly) corresponds to the failure probability. Note that we should not interpret it as such as it does not make sense to add probabilities while those feature will be added in the global `cost function` of the min cost flow solvers. 

This results in a `unit` cost of `1/(c+1)`. since integers are preferable we multiply the unit cost with the maximum observed channel `capacity` of all channels and ignoring the decimals. 

Thus the **final integer linearized uncertainty unit cost** is: `int(c_{max}/(c+1))`

## (routing) fees

The routing `fee` is a feature that usually corresponds directly to the fees that are being charged to forward a payment of size `a` along a channel. According to the protocol it is currently being computed as: 

`fee(a) = a*ppm/1000 + base_fee_msat`

## linearized fees

As discussed in our research and on the mailinglist the fee function of the lightning network is not linear. In the mentioned discussion one can see that `fee(a+b) = fee(a)+fee(b) - base_fee_msat` which is linear if and only if `base_fee_msat = 0`. 

Thus in computation we currently linearize the `fee` function by ignoring the `base fee` so we use `fee(a) = a*ppm/1000` giving us a `unit cost` of `ppm/1000` 

As min cost flow solvers prefer integer cost we scale by `1000` and arrive at the **final integer linearized fee unit cost** which is just the: `ppm`. 

For this to work good enough we might also take channel that charge a very small `base_fee` and later just pay it when we send the onions.


## Features

The `features` of our cost function encode our optimization goals. Typicalfeatures goals might be to have a cheap price (usually encoded by `fee`) or to have a high reliability (in this work best encoded by the `uncertainty cost`that comes from the estimated `success probability` that the channel has enough `liquidity`. Other features that seam reasonable are things related to `latency` of payments (e.g. [pyhsical geodistance or virtaul IP-distance as indicted in this comment](https://github.com/lightningdevkit/rust-lightning/issues/1170)). Implementations have also experimented with other features like CLTV delta or channel age. 

Feature engineering is a crucial part for future research 

## Success Probability

The definition of the `success probability` is in our case modelled by estimating the likelihood that a channel with a `capacity` of `c` has `a` sats of `liquidity` available. 

In the base case where we have not learnt anything about the channel's `liquidity` in the past the `success probability` for making a payment of size `a` stays at `P(X>=a) = (c+1-a)/(c+1)`

Let's say a payment delivery algorithm sends an onion via a path of channel and wants to add an HTLC with amount `h` to a channel of capacity `c` there are several cases that we can observe to update our uncertainty about the network for future payments of size `a`

1. Payment failed at the channel: This means the channel of capacity `c` has at most `h-1` satoshi in it. In Terms of Probability theory we can ask ourselves what is the success rate for a subsequent payment of size `a` given the event `X<h` (the htlc  `h` has failed) which can be expressed as `P(X>=a | X<h)`. Here we see because of the condition that for `a>=h` the proabbility has to be `0`. Computing the conditional probability for the remainder in the uniform case we can se `P(X>=a | X < h) = (h-a)/h`. Note that this is the same as the success probability for a channel of capacity `h-1`. 
2. Payment failed at a downstream channel: This means that the channel of capcity `c` was able to route `h` satoshi and has at least a minimum `liquidity` of `h`. This means that `P(X>=a | X>=h) = 1`  for all amounts `a <=h` and in the unform case for all larger values of `a` the conditional probability materializes to: `P(X>=a)/P(X>=h) = ((c+1-a)/(c+1))/((c+1-h)/(c+1)) = (c+1-a)/(c+1-h)` Note that this fraction is always smaller than `1` as `a >=h`
3. The case that the payment was successfull or did not return an error: Following the same thoughts and the first bullet point at the end of section 3 in the second paper we know that for a future payment of size `a` we have to look at `P(X>=a + h | X >=h)` which in the uniform case materializes to `( (c-h) + 1 - a)/( (c - h) + 1)`. Note that this is the same as if the channel shrunk from `c` to `c-h` as we now know that the maximum `liquidity` is not `c` but rather `c-h`


## optimal payment flow / optimal mpp split

We call a payment flow optimal if it minimizes the `cost function` that encodes our optimization goals. Note that the solution of the min cost flow problem also defines a split of the payment across various paths. While simple algorithms for discecting a flow into paths exist the actual disection is not uique and needs further research. Especially when we start taking channel configurations into account.

## Payment Loop

The payment loop is the trial and error process of delivering a certain amount of satoshis from a sending node to a recipient. This is usually done by generating a set of candidate onions from the solution of an `optimal payment flow`. The onions are concurrently being sent out and error are being used to update our belief about the `liquidity` in the `uncertainty network`.

## Probabilistic Payment Delivery

The idea of `probabilistic payment delivery` is to send onions that maximize the success proabbility for all (or parts of them) to be successul. This takes the `uncertainty` about the remote `liquidity` into consideration and was first introduced in path finding (Note I did not use the term path findng nore did I put it in the glossary)

## Pickhardt Payments
As the term is more and more starting to flow around I will try to summarize best what is currently from a technial point of view meant by it: 

`Using probabilistic payment delivery` in a round based `payment loop` that updeates our belief of the remote `liquidity` in the `uncertainty network` and generates reliable and cheap `payment flows` in every round by solving a `piece wise linearized min integer cost flow problem with a seperable cost function` (I start to see why a shorter term was needed).

In [1]:
from ortools.graph import pywrapgraph
import time
import random
import networkx as nx
import json
from math import log2 as log

In [2]:
def next_hop(path):
    """
    generator to iterate through edges indext by node id of paths
    
    The path is a list of node ids. Each call returns a tuple src, dest of an edge in the path    
    """
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)

In [3]:
class Channel:
    """
    The channel class contains basic information of a channel that will be used to create the
    Uncertainty network.
    
    Since we optimize for reliability via a probability estimate for liquidity that is based
    on the capacity of the channel the class contains the `capacity` as seen in the funding tx output.
    
    As we also optimize for fees and want to be able to compute the fees of a flow the classe
    contains information for the feerate (`ppm`) and the base_fee (`base`). 
    
    Most importantly the class stores our belief about the liquidity information of a channel.
    This is done by reducing the uncertainty interval from [0,`capacity`] to 
    [`min_liquidity`, `max_liquidity`].
    Additionally we need to know how many sats we currently have allocated via outstanding onions
    to the channel which is stored in `inflight`.
    
    the `key` field is necessary to map the channel to its corresponding arcs in the minc cost flow solver
    
    FIXME: the class does not contain min_htlc_size, channel_reserve or other meta data from gossip yet.
    """
    def __init__(self,size,key,ppm,base):
        self.__key=key
        self.__ppm=ppm
        self.__base=base
        self.__capacity=size
        self.__minLiquidity=0
        self.__maxLiquidity=size
        self.__inflight = 0
        
    def __str__(self):
        return "Size: {} with {:4.2f} bits of Entropy. Uncertainty Interval: [{},{}]".format(self.__capacity, self.entropy(), self.__minLiquidity, self.__maxLiquidity)

    def get_key(self):
        return self.__key
    
    def get_capacity(self):
        return self.__capacity
    
    def get_ppm(self):
        return self.__ppm
    
    def get_base(self):
        return self.__base
    
    def get_max_liquidity(self):
        return self.__maxLiquidity

    def get_min_liquidity(self):
        return self.__minLiquidity
    
    def forget_information(self):
        """
        resets the information that we belief to have about the channel. 
        """
        self.__minLiquidity=0
        self.__maxLiquidity=self.__capacity
        self.__inflight = 0
        
    def set_min_liquidity(self,value):
        self.__minLiquidity=value
    
    def set_max_liquidity(self,value):
        self.__maxLiquidity=value

    def allocate_amount(self,amt):
        self.__inflight += amt
        return
    
    def entropy(self):
        """
        returns the uncertainty that we have about the channel
        
        FIXME: Do we have to respect inflight information? I assume no.
        """
        return log(self.__maxLiquidity - self.__minLiquidity + 1)
    
    def success_probability(self,amt):
        """
        returns the estimated success probability for a payment based on our belief about the channel using a uniform distribution.

        In particular the conditional probability P(X>=a | min_liquidity < X < max_liquidity)
        is computed based on our belief also respecting how many satoshis we have currently 
        outstanding and allocated.
        
        FIXME: Potentially test other prior distributions like mixedmodels where most funds are on one side of the channel
        """
        tested_liquidity = amt + self.__inflight

        if tested_liquidity <= self.__minLiquidity:
            return 1.0
        elif amt >= self.__maxLiquidity:
            return 0
        else:            
            conditional_amount = tested_liquidity - self.__minLiquidity
            conditional_capacity = self.__maxLiquidity - self.__minLiquidity
            return float(conditional_capacity + 1 - conditional_amount)/ (conditional_capacity + 1)
      
        
        """
        This is how it would be computed without respecting in_flight information 
        
        if amt <= self.__minLiquidity:
            return 1.0
        elif amt >= self.__maxLiquidity:
            return 0
        else:
            
            effective_amount = amt - self.__minLiquidity
            effective_capacity = self.__maxLiquidity - self.__minLiquidity
            return float(effective_capacity+1 - effective_amount)/ (effective_capacity+1)
        """


In [4]:
class OracleChannel(Channel):
    """
    The Oracle channel inherits from the channel and is used for our simulation to act as the Oracle
    
    The Oracle channel has assigned liquidity information about a payment channel and can be tested 
    against. Note that on mainnet the actual channels act as oracles as one tests them by sending out
    onions. For the simulation we just create ourselves an `known_liquidity_network` as an oracle.
    (Note in the optimally reliable payment flows paper we still called this the known balance graph
    however we retracted from using the term balance.)
    
    
    """
    def __init__(self,size,key,ppm,base,actual_liquidity=None):
        super().__init__(size,key,ppm,base)
        self.__actual_liquidity=actual_liquidity
        if actual_liquidity is None or actual_liquidity >= size or actual_liquidity < 0:
            self.__actual_liquidity = random.randint(0,size)
    def __str__(self):
        return super().__str__()+" actual Liquidity: {}".format(self.__actual_liquidity)
    
    def update_knowledge(self,amt):
        """
        updates our knowledge about the channel if we tried to probe it for amount `amt`
        """
        if amt <= self.__actual_liquidity:
            self.set_min_liquidity(max(self.get_min_liquidity(),amt))
            return True
        else:
            self.set_max_liquidity(min(self.get_max_liquidity(), amt))
            return False
    
    #needed for BOLT14 test experiment
    def learn_n_bits(self,n):
        """
        conducts n probes of channel via binary search starting from our belief
        
        This of course only learns `n` bits if we use a uniform success probability as a prior
        thus this method will not work if a different prior success probability is assumed 
        """
        if n <= 0:
            return
        amt = self.get_min_liquidity() + int((self.get_max_liquidity() - self.get_min_liquidity())/2)
        self.update_knowledge(amt)
        self.learn_n_bits(n-1)
    
    def get_actual_liquidity(self):
        """
        Tells us the actual liquidity according to the oracle. 
        
        This is usful for experiments but must of course not be used in routing and is also
        not a vailable if mainnet remote channels are being used.
        """
        return self.__actual_liquidity

In [5]:
class Oracle:
    def __init__(self,):
        return 
    
    def get_max_cap(self):
        """
        returns the maximum capacity on the channel graph
        
        This is necessary to produce integer costs for the linearized uncertainty cost
        """
        return self.__max_cap
    
    def reset_uncertainty_network(self):
        """
        Forgets all learnt information on the uncertainty network. 
        
        This is usful to resuse the same data from the oracle between various experiments
        """
        for key in self.__arcs.keys():
            self.__arcs[key].forget_information()
    
    def __get_channel_json(self,fp):
        #$ lightning-cli listchannels > listchannels20211028.json
        f = open(fp)
        return json.load(f)["channels"]
    
    def __prepare_integer_indices_for_nodes(self,channels):
        """
        necessary for the OR-lib by google and the min cost flow solver
        """
        # let's first find all node_ids 
        # so that we can build the look up table and use integer unit costs

        node_ids = set()
        for c in channels:
            src = c["source"]
            dest = c["destination"]
            node_ids.add(src)
            node_ids.add(dest)        

        # let's initialize the look up tables for node_ids to integers from [0,...,#number of nodes]
        # this is necessary because of the API of the Google Operations Research min cost flow solver
        self.__node_key_to_id = {}
        self.__id_to_node_key = {}
        for k, node_id in enumerate(node_ids):
            self.__node_key_to_id[node_id]=k
            self.__id_to_node_key[k]=node_id
    
    def import_channels(self,fp):
        """
        This function does all the magic! 
        
        It starts by importing the channel_graph from c-lightning listchannels command and then 
        it goes through the channels to find all node ids and max capacity (to make integer uncertainty cost)
        in a second pass it goes over all channels and adds arcs to the modelled linearized network
        for each channel N arcs are being added with increasing unit costs to mimic the convex behavior
        the piecewise dissection is not optimal nor is the linear approximation of negative log probs exact
        however this does not matter for the sake of argument the runtime will not change if the costs
        are chosen a bit more optimally. But the code will blow up thus those simplifications
        """

        channels = self.__get_channel_json(fp)
        self.__prepare_integer_indices_for_nodes(channels)

        self.__max_cap = 0
        for c in channels:
            cap = c["satoshis"]
            if cap > self.__max_cap:
                self.__max_cap = cap

        self.__arcs = {}
        self.__parallel = {}
        for c in channels:
            cap = c["satoshis"]
            sid = c["short_channel_id"]

            src = self.__node_key_to_id[c["source"]]
            dest = self.__node_key_to_id[c["destination"]]
            cap = c["satoshis"]

            key = "{}x{}x{}".format(src,dest,sid)
            base = c["base_fee_millisatoshi"]
            ppm = c["fee_per_millionth"]
            self.__arcs[key]=OracleChannel(cap,key,ppm,base)
            arc = "{}x{}".format(src,dest)
            if arc in self.__parallel:
                self.__parallel[arc].append(key)
            else:
                self.__parallel[arc] = [key]


    def arcs_to_networkx(self,base_threshold = None):
        """
        stores the arcs also in networkx for fee computation and display of probabilities
        
        FIXME: it should actually be the other way around. First store the channel graph in 
        networkx and then compute the piece wise linearized arcs from this
        """
        self.__channel_graph = nx.MultiDiGraph()
        for channel in self.__arcs.values():
            src, dest, height, index, out = channel.get_key().split("x")
            if base_threshold is not None:
                if channel.get_base() > base_threshold:
                    continue
            self.__channel_graph.add_edge(int(src),int(dest),capacity=channel.get_capacity(),base=channel.get_base(),ppm=channel.get_ppm())
           

                
    def theoretical_maximum_payable_amount(self, source, destination):
        """
        Uses the information from the oracle to compute the min-cut between source and destination
        
        This is only useful for experiments and simulations if one wants to know what would be 
        possible to actually send before starting the payment loop
        """
        self.__channel_graph = nx.MultiDiGraph()
        for channel in self.__arcs.values():
            src, dest, height, index, out = channel.get_key().split("x")
            if base_threshold is not None:
                if channel.get_base() > base_threshold:
                    continue
            self.__channel_graph.add_edge(int(src),int(dest),capacity=channel.get_actual_liquidity(),base=channel.get_base(),ppm=channel.get_ppm())
           
        self.__G = nx.DiGraph()
        for u,v,data in self.__channel_graph.edges(data=True):
            if not self.__G.has_edge(u,v):
                capacity = sum(d.get('capacity') for d in self.__channel_graph.get_edge_data(u,v).values())
                self.__G.add_edge(u, v, capacity=capacity)
        
        mincut, _ = nx.minimum_cut(self.__G, self.__node_key_to_id[source], self.__node_key_to_id[destination])
        return mincut
    
    def get_probability(self,src,dest,amt):
        #FIXME: handle multi edges properly
        #if not self.__channel_graph.has_edge(src,dest):
        capacity = sum(d.get('capacity') for d in self.__channel_graph.get_edge_data(src,dest).values())
        #else:
        #    capacity = self.__channel_graph[src][dest]["capacity"]
        if amt > capacity:
            print(src,dest,self.__channel_graph.get_edge_data(src,dest))
        return float(capacity+1-amt)/(capacity+1)

            
    def get_fees_msat(self,src,dest,amt):
        #FIXME: handle multi edges properly
        ppm = min(d.get('ppm') for d in self.__channel_graph.get_edge_data(src,dest).values())
        base = min(d.get('base') for d in self.__channel_graph.get_edge_data(src,dest).values())
        return int(base + float(amt * ppm)/1000) 
        
    def test_path(self,path,amt):
        probability = 1
        fees = 0
        success = True
        for src,dest in next_hop(path):
            fees += self.get_fees_msat(src,dest,amt)
            probability *= self.get_probability(src,dest,amt)
            key = "{}x{}".format(src,dest)
            #FIXME: do not just take the first parallel channel
            arc_key = self.__parallel[key][0]
            if success:
                success = self.__arcs[arc_key].update_knowledge(amt)
            
        return success, fees, probability
    
    def allocate_path(self,path,amt):
        t,_,_ = self.test_path(path,amt)
        if t == False:
            return False
        for src,dest in next_hop(path):
            key = "{}x{}".format(src,dest)
            #FIXME: do not just take the first parallel channel
            arc_key = self.__parallel[key][0]
            if success:
                success = self.__arcs[arc_key].allocate_amount(amt)
        
        
    def get_arcs(self):
        return self.__arcs
    
    def look_up_id(self,node_id):
        return self.__node_key_to_id[node_id]
    
    def get_uncertainty_network(self,start_node,mu,threshold_base =None):
        #FIXME: find mu
        start_node = self.__node_key_to_id[start_node]

        max_cap = oracle.get_max_cap()
        used_arcs = []
        for key, channel in self.__arcs.items():
            base = channel.get_base()
            if threshold_base is not None:
                if base > threshold_base:
                    continue

            cap = channel.get_capacity()
            src,dest,_,_,_ = channel.get_key().split("x")
            src = int(src)
            dest = int(dest)

            is_own_channel = False
            if src == start_node:
                is_own_channel=True
                
            #FIXME: might need a fix if we account for HTLCs in flight but I guess this is just reduced from actual liqudity
            if is_own_channel: #we know exactly how much liquidity we have in our channels
                channel.set_min_liquidity(channel.get_actual_liquidity())
                channel.set_min_liquidity(channel.get_actual_liquidity())

            conditional_capacity = channel.get_max_liquidity() - channel.get_min_liquidity()
            min_liquidity = channel.get_min_liquidity()
            ppm = channel.get_ppm()
            
            if is_own_channel: #we don't have to pay ppm on our own channels
                ppm = 0
            
            n = N
            
            #FIXME: prune expensive and unlikeli channels
            #FIXME: compute smarter linearization eg: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf            
            #using certainly available liquidity costs us nothing but fees
            if int(min_liquidity/QUANTIZATION) > 0:
                used_arcs.append((src,dest,int(min_liquidity/QUANTIZATION),0 + mu*ppm))
                n-=1

            # FIXME: include the 
            if int(conditional_capacity/QUANTIZATION) > 0:
                unit_cost = int(max_cap/conditional_capacity)
                for i in range(n):
                    #arc format is src, dest, capacity, unit_cost
                    # THIS IS THE IMPORTANT LINE OF CODE WHERE THE MAGIC HAPPENS
                    used_arcs.append((src,dest,int(conditional_capacity/(N*QUANTIZATION)),(i+1)*unit_cost +mu* ppm))
        return used_arcs
    
    ##############################################################
    #
    #     needed for FOAF liquidity sharing  (BOLT 14 proposal)  
    #
    ##############################################################

    def entropy(self):
        return sum(arc.entropy() for _,arc in self.__arcs.items())
    
    
    def activate_network_wide_uncertainty_reduction(self,n):
        for arc in self.__arcs.values():
            arc.learn_n_bits(n)
            
    def activate_foaf_uncertainty_reduction(self,src,dest):
        ego_netwok=set()
        foaf_network = set()

        out_set = set()
        edges = self.__channel_graph.out_edges(self.__node_key_to_id[src])
        for edge in edges:
            ego_netwok.add("{}x{}".format(edge[0],edge[1]))
            out_set.add(edge[1])
        
        for node in out_set:
            edges = self.__channel_graph.out_edges(node)
            for edge in edges:
                foaf_network.add("{}x{}".format(edge[0],edge[1]))

        in_set = set()
        edges = self.__channel_graph.in_edges(self.__node_key_to_id[dest])
        for edge in edges:
            ego_netwok.add("{}x{}".format(edge[0],edge[1]))
            in_set.add(edge[0])
        
        for node in in_set:
            edges = self.__channel_graph.out_edges(node)
            for edge in edges:
                foaf_network.add("{}x{}".format(edge[0],edge[1]))
                
                
        #print(len(ego_netwok))
        for k,arc in self.__arcs.items():
            #print(k)
            vals = k.split("x")
            key = "{}x{}".format(vals[0],vals[1])
            if key in ego_netwok:
                l = arc.get_actual_liquidity()
                arc.update_knowledge(l-1)
                arc.update_knowledge(l+1)
                #print(key,arc.entropy())
            
            if key in foaf_network:
                arc.learn_n_bits(2)
                l = arc.get_actual_liquidity()
                arc.update_knowledge(l-1)
                arc.update_knowledge(l+1)
                #print(key, arc.entropy())
        print("channels with full knowlege: ", len(ego_netwok))
        print("channels with 2 Bits of less entropy: ", len(foaf_network))

# Define some helper functions to prepare experiments

In [6]:
def prepare_mcf_solver(src, dest,amt, mu=0, base=0):
    """
    computes the uncertainty network given our prior belief and prepares the min cost flow solver
    
    This function can define a value for \mu to control how heavily we combine the uncertainty cost and fees
    Also the function supports only taking channels into account that don't charge a base_fee higher or equal to `base`

    returns the instantiated min_cost_flow object from the google OR-lib that contains the piecewise linearized problem
    """
    used_arcs = oracle.get_uncertainty_network(src,mu,base)
    oracle.arcs_to_networkx(base)
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each of the prepared arcs from import_channels().
    node_ids = set()
    for arc in used_arcs:
        node_ids.add(arc[0])
        node_ids.add(arc[1])
        min_cost_flow.AddArcWithCapacityAndUnitCost(arc[0], arc[1], arc[2],
                                                    arc[3])

    # Add node supply to 0 for all nodes
    for i in node_ids:
        min_cost_flow.SetNodeSupply(i, 0)

    #add amount to sending node
    min_cost_flow.SetNodeSupply(oracle.look_up_id(src),int(amt/QUANTIZATION))

    #add -amount to recipient nods
    min_cost_flow.SetNodeSupply(oracle.look_up_id(dest),-int(amt/QUANTIZATION))
    return min_cost_flow



In [7]:
def disect_flow_to_paths(min_cost_flow,s,d):
    """
    A standard algorithm to disect a flow into several paths.
    
    FIXME: Note that this disection while accurate is probably not optimal in practise. 
    As noted in our Probabilistic payment delivery paper the payment process is a bernoulli trial 
    and I assume it makes sense to disect the flow into paths of similar likelihood to make most
    progress but this is a mere conjecture at this point. I expect quite a bit of research will be
    necessary to resolve this issue.
    """
    total_flow = {}

    #first collect all linearized edges which are assigned a non zero flow
    for i in range(min_cost_flow.NumArcs()):
        if min_cost_flow.Flow(i) == 0:
            continue
        src = min_cost_flow.Tail(i)
        dest = min_cost_flow.Head(i)
        flow = min_cost_flow.Flow(i)*QUANTIZATION

        key = str(src)+":"+str(dest)
        if key in total_flow:
            total_flow[key]=(src,dest,total_flow[key][2]+flow)
        else:
            total_flow[key]=(src,dest,flow)
    
    
    G = nx.DiGraph()
    for key, val in total_flow.items():
        src, dest, flow = val
        G.add_edge(src,dest,weight=flow)

    m = 1
    paths = []
    while m>0:
        try: 
            path = nx.shortest_path(G,s,d)
            m = min(G[src][dest]["weight"] for src, dest in next_hop(path))
            #t+=m
            #print("{} sats along {}".format(m*10000,path_str(path)))
            paths.append((path,m))
            for src,dest in next_hop(path):
                G[src][dest]["weight"]-=m
                if G[src][dest]["weight"]==0:
                    G.remove_edge(src,dest)
        except: 
            break
    return paths



In [8]:
def make_attempt(src, dest, amt,mu,base):
    """
    computes the optimal payment split to deliver `amt` from `src` to `dest` and updates our belief about the liquidity
    
    This is one step within the payment loop.
    
    Retuns the residual amount of the `amt` that could ne be delivered and the paid fees
    (on a per channel base not including fees for downstream fees) for the delivered amount
    
    the function also prints some results an statistics about the paths of the flow to stdout.
    """
    start = time.time()

    mcf = prepare_mcf_solver(src,dest,amt,mu,base)   

    status = mcf.Solve()


    if status != mcf.OPTIMAL:
        print('There was an issue with the min cost flow input.')
        print(f'Status: {status}')
        exit(1)

    paths = disect_flow_to_paths(mcf,oracle.look_up_id(src),oracle.look_up_id(dest))
    #print(paths)    

    end = time.time()
    print("\\mu: {} \nRuntime of flow computation: {:4.2f} sec ".format(mu, end-start))

    total_fees = 0
    paid_fees = 0
    residual_amt = 0
    number_failed_paths = 0
    for path in paths:
        success, fee, probability = oracle.test_path(path[0],path[1])
        fee /= 1000.
        total_fees += fee
        print("Success: {} \t fee: {:8.3f}msat \t p = {:5.2f}%   amt: {:7}sats path hops: {}".
              format(success, fee, probability*100, path[1], len(path[0])))
        if success == False:
            number_failed_paths += 1
            residual_amt += path[1]
        else:
            paid_fees += fee

    print("total_fee: {:8.3f} msat \t paid fees: {:8.3f} msat".format(total_fees, paid_fees))
    return residual_amt, paid_fees, len(paths), number_failed_paths



In [9]:
def run_pickhardt_payments_experiment(oracle,src,dest,amt,mu=1,base=0):
    """
    conduct one experiment! might need to call oracle.reset_uncertainty_network() first
    I could not put it here as some experiments require sharing of liqudity information

    """
    entropy_start=oracle.entropy()
    start = time.time()
    full_amt = amt
    cnt = 0
    total_fees = 0
    number_number_of_onions = 0
    total_number_failed_paths = 0
    while amt > 0 and cnt < 10:
        print("\nTry: ", amt, " sats. Round number: ", cnt+1)
        amt, paid_fees, num_paths, number_failed_paths = make_attempt(src,dest,amt,mu,base)
        number_number_of_onions += num_paths
        total_number_failed_paths+=number_failed_paths
        total_fees += paid_fees
        cnt+=1
    end = time.time()
    entropy_end = oracle.entropy()
    print("\nSUMMARY:")
    print("Rounds of mcf-computations: ", cnt)
    print("Number of onions sent: ", number_number_of_onions)
    print("Number of failed onions: ", total_number_failed_paths, " failure rate: {:4.2f}% ".format(total_number_failed_paths*100./number_number_of_onions))
    print("total runtime (including inefficient memory managment): {:4.3f} sec".format(end-start))
    print("Learnt entropy: {:5.2f} bits".format(entropy_start-entropy_end))
    print("Fees for successfull delivery: {:8.3f} sat --> {} ppm".format(total_fees,int(total_fees*1000*1000/full_amt)))
    
    

# Conducting experiments

We now use our sequential simulation of the uncertainty network to coduct a few experiments. In all cases we try to deliver Payments from Rene's node to Carsten Otto (who is working on an alternative implementation) on a recent snapshot of the channel graph.

In the experiments we first want to study a few things: 

1. What happens if we mainly optimize for high success probability
2. What happens if we optimize for both success probability and fees
3. what happens if we optimize mainly for fees
4. How much better is the delivery of payments if we globaly would know 2 bits of information on alle channels
5. What is the effect of the BOLT 14 proposal to reliability where nodes would share liquidity information of their direct peers
6. MISSING: How is the difference in paid fee and payment attempts with different focus on various features (selection of \mu)

In [10]:
#Rene Pickhardt's public node_key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"

AMT = 50*1000*1000
N = 5
QUANTIZATION=10000

In [11]:
oracle = Oracle()
oracle.import_channels("listchannels20211028.json")

#we only conduct the computation on the channels that have a basefee of 0
oracle.arcs_to_networkx(0)
oracle.activate_foaf_uncertainty_reduction(RENE,C_OTTO)

channels with full knowlege:  215
channels with 2 Bits of less entropy:  8750


In [12]:

print("OPTIMIZE FOR soly for RELIABILITY")
oracle.reset_uncertainty_network()
run_pickhardt_payments_experiment(oracle,RENE,C_OTTO,AMT,mu=0,base=0)


OPTIMIZE FOR soly for RELIABILITY

Try:  50000000  sats. Round number:  1
\mu: 0 
Runtime of flow computation: 1.43 sec 
Success: True 	 fee: 6153.950msat 	 p = 64.05%   amt: 3350000sats path hops: 3
Success: True 	 fee: 2152.570msat 	 p = 29.87%   amt: 1610000sats path hops: 3
Success: True 	 fee: 3374.000msat 	 p = 69.33%   amt: 2000000sats path hops: 3
Success: True 	 fee:  312.290msat 	 p = 89.94%   amt:  170000sats path hops: 3
Success: False 	 fee: 12752.550msat 	 p = 47.10%   amt: 7650000sats path hops: 3
Success: True 	 fee:   18.370msat 	 p = 96.10%   amt:   10000sats path hops: 4
Success: True 	 fee: 3248.960msat 	 p = 43.40%   amt: 1430000sats path hops: 4
Success: True 	 fee:  949.000msat 	 p = 87.99%   amt:  500000sats path hops: 4
Success: True 	 fee: 1423.500msat 	 p = 22.80%   amt:  750000sats path hops: 4
Success: True 	 fee: 2738.630msat 	 p = 28.11%   amt: 1370000sats path hops: 4
Success: True 	 fee: 4052.290msat 	 p = 82.72%   amt: 1870000sats path hops: 4
Success:

In [13]:
print("\n\nOTPTIMIZE RELIABILITY AND FEES")
oracle.reset_uncertainty_network()
run_pickhardt_payments_experiment(oracle,RENE,C_OTTO,AMT,mu=1,base=0)






OTPTIMIZE RELIABILITY AND FEES

Try:  50000000  sats. Round number:  1
\mu: 1 
Runtime of flow computation: 1.23 sec 
Success: False 	 fee: 14652.900msat 	 p = 16.08%   amt: 10050000sats path hops: 3
Success: True 	 fee: 6631.570msat 	 p = 61.60%   amt: 3610000sats path hops: 3
Success: True 	 fee: 2060.480msat 	 p = 23.93%   amt: 1370000sats path hops: 3
Success: True 	 fee: 1537.000msat 	 p = 64.00%   amt: 1000000sats path hops: 3
Success: True 	 fee:  908.700msat 	 p = 23.40%   amt:  650000sats path hops: 3
Success: True 	 fee: 2152.570msat 	 p = 29.87%   amt: 1610000sats path hops: 3
Success: True 	 fee: 6748.000msat 	 p = 44.00%   amt: 4000000sats path hops: 3
Success: True 	 fee: 1153.600msat 	 p = 60.00%   amt:  800000sats path hops: 3
Success: True 	 fee:  312.290msat 	 p = 89.94%   amt:  170000sats path hops: 3
Success: True 	 fee: 3454.770msat 	 p = 75.12%   amt: 2090000sats path hops: 4
Success: True 	 fee:   18.370msat 	 p = 96.10%   amt:   10000sats path hops: 4
Success:

In [14]:
    
print("\n\nOTPTIMIZE MAINLY FOR FEES")
oracle.reset_uncertainty_network()
run_pickhardt_payments_experiment(oracle,RENE,C_OTTO,AMT,mu=100,base=0)





OTPTIMIZE MAINLY FOR FEES

Try:  50000000  sats. Round number:  1
\mu: 100 
Runtime of flow computation: 1.28 sec 
Success: True 	 fee: 8354.340msat 	 p = 43.36%   amt: 5730000sats path hops: 3
Success: True 	 fee: 1338.560msat 	 p = 46.83%   amt:  890000sats path hops: 3
Success: True 	 fee: 1089.600msat 	 p = 44.00%   amt:  800000sats path hops: 3
Success: False 	 fee: 1087.200msat 	 p = 10.57%   amt:  800000sats path hops: 3
Success: False 	 fee: 3061.730msat 	 p =  6.48%   amt: 2290000sats path hops: 3
Success: False 	 fee: 4037.600msat 	 p =  3.75%   amt: 2800000sats path hops: 3
Success: True 	 fee:  318.340msat 	 p = 47.75%   amt:  220000sats path hops: 4
Success: False 	 fee:  230.080msat 	 p = 51.83%   amt:  160000sats path hops: 4
Success: False 	 fee:  386.370msat 	 p = 45.34%   amt:  270000sats path hops: 4
Success: False 	 fee:  675.000msat 	 p = 44.44%   amt:  500000sats path hops: 4
Success: True 	 fee:  409.200msat 	 p = 66.46%   amt:  300000sats path hops: 4
Success:

\mu: 100 
Runtime of flow computation: 1.27 sec 
Success: True 	 fee: 1934.040msat 	 p = 15.27%   amt: 1420000sats path hops: 3
Success: False 	 fee:  312.570msat 	 p = 67.54%   amt:  230000sats path hops: 3
Success: False 	 fee: 2553.670msat 	 p = 19.09%   amt: 1910000sats path hops: 3
Success: True 	 fee:  243.000msat 	 p = 76.76%   amt:  180000sats path hops: 4
Success: False 	 fee: 3037.260msat 	 p = 14.54%   amt: 2270000sats path hops: 4
Success: True 	 fee:   66.900msat 	 p = 78.60%   amt:   50000sats path hops: 4
Success: True 	 fee:  256.310msat 	 p = 19.19%   amt:  190000sats path hops: 4
Success: True 	 fee:  382.200msat 	 p = 76.52%   amt:  280000sats path hops: 4
Success: False 	 fee: 1206.000msat 	 p = 29.14%   amt:  900000sats path hops: 4
Success: False 	 fee:   82.200msat 	 p = 85.37%   amt:   60000sats path hops: 5
Success: True 	 fee:  203.550msat 	 p = 75.85%   amt:  150000sats path hops: 5
Success: True 	 fee:  818.400msat 	 p = 18.28%   amt:  600000sats path hops: 

In [15]:
    
print("\n\nOTPTIMIZE MAINLY FOR FEES use global uncertainty reduction")
oracle.reset_uncertainty_network()
oracle.activate_network_wide_uncertainty_reduction(2)
run_pickhardt_payments_experiment(oracle,RENE,C_OTTO,AMT,mu=100,base=0)





OTPTIMIZE MAINLY FOR FEES use global uncertainty reduction

Try:  50000000  sats. Round number:  1
\mu: 100 
Runtime of flow computation: 1.69 sec 
Success: False 	 fee: 15848.460msat 	 p = 12.40%   amt: 10870000sats path hops: 3
Success: True 	 fee: 2060.480msat 	 p = 23.93%   amt: 1370000sats path hops: 3
Success: True 	 fee: 1921.250msat 	 p = 56.25%   amt: 1250000sats path hops: 3
Success: True 	 fee: 1484.580msat 	 p = 28.97%   amt: 1090000sats path hops: 3
Success: False 	 fee:  231.030msat 	 p = 75.48%   amt:  170000sats path hops: 3
Success: False 	 fee: 2353.120msat 	 p = 24.39%   amt: 1760000sats path hops: 3
Success: False 	 fee: 1442.000msat 	 p = 51.56%   amt: 1000000sats path hops: 3
Success: True 	 fee:  100.660msat 	 p = 22.97%   amt:   70000sats path hops: 4
Success: True 	 fee:  499.500msat 	 p = 56.27%   amt:  370000sats path hops: 4
Success: True 	 fee: 4674.330msat 	 p = 14.77%   amt: 3110000sats path hops: 4
Success: False 	 fee:  512.240msat 	 p = 40.68%   amt:

\mu: 100 
Runtime of flow computation: 1.13 sec 
Success: True 	 fee: 1484.580msat 	 p = 28.97%   amt: 1090000sats path hops: 3
Success: True 	 fee:  217.440msat 	 p = 76.84%   amt:  160000sats path hops: 3
Success: False 	 fee: 2179.310msat 	 p = 29.13%   amt: 1630000sats path hops: 3
Success: True 	 fee:  100.660msat 	 p = 22.97%   amt:   70000sats path hops: 4
Success: True 	 fee:  499.500msat 	 p = 56.27%   amt:  370000sats path hops: 4
Success: True 	 fee:  350.480msat 	 p = 56.01%   amt:  260000sats path hops: 4
Success: False 	 fee: 2542.200msat 	 p = 24.06%   amt: 1900000sats path hops: 4
Success: True 	 fee:   93.660msat 	 p = 70.58%   amt:   70000sats path hops: 4
Success: True 	 fee:  256.310msat 	 p = 19.19%   amt:  190000sats path hops: 4
Success: True 	 fee:  172.680msat 	 p = 66.35%   amt:  120000sats path hops: 4
Success: False 	 fee:  938.000msat 	 p = 41.95%   amt:  700000sats path hops: 4
Success: True 	 fee:  804.720msat 	 p = 13.16%   amt:  560000sats path hops: 5


In [16]:
print("\n\nOTPTIMIZE MAINLY FOR FEES use foaf uncertainty reduction")
oracle.reset_uncertainty_network()
oracle.arcs_to_networkx(0)
oracle.activate_foaf_uncertainty_reduction(RENE,C_OTTO)
run_pickhardt_payments_experiment(oracle,RENE,C_OTTO,AMT,mu=1,base=0)




OTPTIMIZE MAINLY FOR FEES use foaf uncertainty reduction
channels with full knowlege:  215
channels with 2 Bits of less entropy:  8750

Try:  50000000  sats. Round number:  1
\mu: 1 
Runtime of flow computation: 0.91 sec 
Success: True 	 fee: 13180.320msat 	 p = 21.27%   amt: 9040000sats path hops: 3
Success: True 	 fee: 2060.480msat 	 p = 23.93%   amt: 1370000sats path hops: 3
Success: True 	 fee: 2197.910msat 	 p = 50.98%   amt: 1430000sats path hops: 3
Success: True 	 fee: 1511.820msat 	 p = 28.04%   amt: 1110000sats path hops: 3
Success: True 	 fee:  217.440msat 	 p = 76.84%   amt:  160000sats path hops: 3
Success: True 	 fee: 2152.570msat 	 p = 29.87%   amt: 1610000sats path hops: 3
Success: True 	 fee: 5971.980msat 	 p = 49.35%   amt: 3540000sats path hops: 3
Success: True 	 fee: 1225.700msat 	 p = 57.83%   amt:  850000sats path hops: 3
Success: True 	 fee:  289.180msat 	 p = 78.79%   amt:  190000sats path hops: 4
Success: True 	 fee:  129.420msat 	 p =  6.99%   amt:   90000sat